In [3]:
import pandas as pd
import glob
import os
import re

# 메인 폴더 경로
base_folder_path = "C:/Users/jungi/startcoding/실전편/2/2023 선수별/"

# 팀 목록 (팀 폴더 이름)
teams = ['강원', '광주', '수원FC', '대구', '대전','서울', '수원', '울산', '인천', '전북','제주', '포항']

# 사용한 컬럼 목록
columns_to_include = ['포지션', '출전시간(분)', '득점', '인터셉트', '클리어링', '경합 공중 성공', '태클 성공', '유효 슈팅', '키패스', '패스 성공%', '드리블 성공', '슈팅', '크로스 성공']

# 모든 팀에 대한 데이터프레임을 저장할 리스트
all_team_dataframes = []

# 각 팀 폴더를 순회하며 데이터 처리
for team in teams:

    folder_path = os.path.join(base_folder_path, team)
    folder_path = os.path.normpath(folder_path)
    # 특정 패턴과 일치하는 모든 파일 경로 가져오기
    csv_files = glob.glob(os.path.join(folder_path, "2023k리그1_*.csv"))
   
    # 라운드별 데이터프레임을 저장할 딕셔너리
    dataframes = {}

    # 정규표현식 패턴 (예: "2023k리그1_강원_10R _ 전북(A).csv")
    pattern = re.compile(r"2023k리그1_.*_(\d+)R\s.*\.csv")

    # 각 CSV 파일을 읽어서 딕셔너리에 저장
    for file in csv_files:
        match = pattern.match(os.path.basename(file))
        if match:
            round_number = match.group(1)  # 라운드 번호 추출
            dataframes[round_number] = pd.read_csv(file, encoding='cp949')
        else:
            print(f"No match for file: {file}")

    # 메인 데이터프레임 읽기
    main_file_path = os.path.join('C:/Users/jungi/startcoding/실전편/2/', f'2023k리그1_{team}.csv')
    if not os.path.exists(main_file_path):
        print(f"File not found: {main_file_path}")
        continue
    main_df = pd.read_csv(main_file_path, encoding='cp949')

    # 새로운 컬럼을 추가할 준비 (기존 컬럼 이름에 접두어를 추가하여 충돌을 피함)
    for col in columns_to_include:
        main_df[f'Player_Stats_{col}'] = [{} for _ in range(len(main_df))]

    # 각 라운드 데이터프레임에서 선수명과 선택된 컬럼들을 딕셔너리로 만들어 메인 데이터프레임에 추가
    for round_number, df in dataframes.items():
        if '선수명' in df.columns:
            print(df)
            for col in columns_to_include:
                if col in df.columns:
                    if col == '포지션':
                        # 포지션은 문자열로 처리
                        player_stats_dict = dict(zip(df['선수명'], df[col]))
                    else:
                        # 비정상적인 값은 0으로 처리하고 숫자형으로 변환
                        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
                        player_stats_dict = dict(zip(df['선수명'], df[col]))
                    # 디버깅: 각 라운드와 컬럼에 대해 생성된 딕셔너리 출력
                    print(f"Round: {round_number}, Column: {col}, Player Stats Dict: {player_stats_dict}")
                    # 메인 데이터프레임에서 해당 라운드에 맞는 행의 해당 컬럼에 딕셔너리 추가
                    main_df.loc[main_df['Rnd'].astype(str) == str(round_number), f'Player_Stats_{col}'] = main_df.loc[main_df['Rnd'].astype(str) == str(round_number), f'Player_Stats_{col}'].apply(lambda x: player_stats_dict)

    # 팀 이름을 추가한 컬럼을 생성
    main_df['Team'] = team

    # 팀 데이터프레임을 리스트에 추가
    all_team_dataframes.append(main_df)

# 모든 팀 데이터프레임을 하나로 합침
combined_df = pd.concat(all_team_dataframes, ignore_index=True)


#상대팀생성
for i in range(len(combined_df)):
    teams=combined_df.loc[i,'대회'].split('vs')
    if combined_df.loc[i,'Team']==teams[0]:
        combined_df.loc[i,'상대팀'] = teams[1]
    else:
        combined_df.loc[i,'상대팀'] = teams[0]
#득점 실점 기록
for i in range(len(combined_df)):
    temp=combined_df.loc[i,['Rnd','Team','상대팀']]
    our_team=temp[1]
    opponent_team=temp[2]
    Round = temp[0]
    opponent_points = combined_df.loc[(combined_df['Rnd']==Round)&(combined_df['Team']==opponent_team),'득점'].values[0]
    combined_df.loc[i,'실점'] = opponent_points

def result(row):
    if row['득점'] > row['실점']:
        return '승'
    elif row['득점'] == row['실점']:
        return '무'
    else:
        return '패'
combined_df['결과'] = combined_df.apply(result,axis = 1)
combined_df
combined_df.to_csv('2023k1리그종합.csv',encoding = 'cp949', index = False)

    No.    선수명 포지션  등번호 출전시간(분)  득점  도움  슈팅  유효 슈팅  블락된슈팅  ...  클리어링  인터셉트   
0     1    이광연  GK   31     103   0   0   0      0      0  ...     5     3  \
1     2    이웅희  DF    5      89   0   0   0      0      0  ...     9     4   
2     3    김영빈  DF    2     103   0   0   0      0      0  ...    11     4   
3     4    윤석영  DF   20     103   0   0   0      0      0  ...     8     1   
4     5    서민우  MF    4     103   0   0   0      0      0  ...     7     4   
5     6    유인수  MF   17      80   0   0   1      0      0  ...     6     4   
6     7    한국영  MF    8      90   0   0   0      0      0  ...     1     2   
7     8    정승용  MF   22     103   0   0   0      0      0  ...     6     1   
8     9    양현준  FW    7     103   1   0   1      1      0  ...     0     2   
9    10    김대우  FW   14      49   0   0   0      0      0  ...     0     0   
10   11    박상혁  FW   35      49   0   0   0      0      0  ...     0     0   
11   12    김우석  대기   21      14   0   0   0      0      0  ...  